<h3>En esta versión se integra un control tipo dropdown donde se presenta una lista cerrada de diversos terminos.
<BR>Para este ejemplo se toma el rango de los meses de agosto a diciembre de 2021.
<BR>Los terminos del dropdown son a manera de ejemplo, ya que los mismos se pueden agregar o eliminar.
</h3>

In [ ]:
import warnings, json, numpy as np, pandas as pd, dash, plotly.express as px, plotly.graph_objects as go
from textwrap import dedent as d
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash

ruta = 'C://Tweetsentpol.csv'
df = pd.read_csv(ruta, header=0)
df2 = df[['Fecha', 'Polarity']]

fecha_ini = '2021-08-01'
fecha_fin = '2021-12-31'
mask = (df2['Fecha'] >= fecha_ini) & (df2['Fecha'] <= fecha_fin)
df3 = df2.loc[mask]
df3 = df3.groupby(['Fecha']).mean()
df4 = df3.reset_index()
df4['Color'] = "Fecha"

warnings.simplefilter(action='ignore', category=FutureWarning)

# app info
app = JupyterDash(__name__)
styles = {
    'pre': {
        'border': 'thin lightgrey solid',
        'overflowX': 'scroll'
    }
}

# plotly figure
fig = px.line(df4, x="Fecha", y="Polarity", color="Color", title="No label selected")
fig.update_traces(mode="markers+lines")

app.layout = html.Div([
    dcc.Graph(
        id='figure1',
        figure=fig,
    ),
    
    html.Div(
        html.Div([
            dcc.Dropdown(
               options=[
                   {'label': 'COVID', 'value': 'COVID'},
                   {'label': 'Coronavirus', 'value': 'Coronavirus'},
                   {'label': 'Pandemia', 'value': 'Pandemia'},
                   {'label': 'Vacuna', 'value': 'Vacuna'},
                   {'label': 'Cubrebocas', 'value': 'Cubrebocas'},
                   {'label': 'AMLO', 'value': 'AMLO'},
                   {'label': 'Pfizer', 'value': 'Pfizer'},
                   {'label': 'AstraZeneca', 'value': 'AstraZeneca'},
                   {'label': '*** Todos ***', 'value': 'Todos'},
               ], placeholder="Seleccione un termino para filtrado", id='comboterms', style={"width": "60%"}),
            
            html.Pre(id='hoverdata2', style=styles['pre']),
    ]), )
])

ruta2 = 'C://Tweets.csv'
dftuits = pd.read_csv(ruta2, header=0)

<h3>Idealmente, primero se debe seleccionar un punto de la serie de tiempo para mostrar los tweets de ese día. Los cuales se identifican por el prefijo "Tweet-> ".
<BR>Posteriormente y de manera opcional, se puede seleccionar un valor de la serie de tiempo.
<BR>También se pueden volver a mostrar todos los tweets originales del día elegido.
</h3>

In [2]:
@app.callback(Output('hoverdata2', 'children'), 
             [Input('figure1', 'clickData'),
              Input('comboterms', 'value')])

def display_hover_data(hoverData, comboterms_name):
    global dftuits2
    button_id = dash.callback_context.triggered[0]['prop_id'].split('.')[0]
    if button_id == 'figure1' and hoverData:
        if hoverData is not None:
            traceref = hoverData['points'][0]['curveNumber']
            pointref = hoverData['points'][0]['pointNumber']

            mask2 = (dftuits['fecha'] == fig.data[traceref]['x'][pointref])
            dftuits2 = dftuits.loc[mask2]
            lostuits = pd.Series("Tweet-> " + dftuits2['tweet'])
            lostuits2 = lostuits.astype(str).str.cat(sep='\n')

            return html.Div([
                    html.Br(),html.Pre('Dia Seleccionado ' + fig.data[traceref]['x'][pointref]),html.Br(),html.Pre(lostuits2)
                ])
        else:
            return 'Ninguno seleccionado'

    elif button_id == 'comboterms' and comboterms_name:
        if comboterms_name is not None:
            if comboterms_name == 'Todos':
                df3 = dftuits2
                lostuits3 = pd.Series("Tweet-> " + df3['tweet'])
                lostuits4 = lostuits3.astype(str).str.cat(sep='\n')
            else:
                df3 = dftuits2[dftuits2['tweet'].str.contains(comboterms_name, case=False)]
                lostuits3 = pd.Series("Tweet-> " + df3['tweet'])
                lostuits4 = lostuits3.astype(str).str.cat(sep='\n')

            return html.Div([
                        html.Pre('Valor Seleccionado ' + comboterms_name),html.Br(),html.Pre(lostuits4)
                    ])
    else:
        raise dash.exceptions.PreventUpdate

app.run_server(mode='external', port = 7078, dev_tools_ui=True,
  dev_tools_hot_reload =True, threaded=True)

Dash app running on http://127.0.0.1:7078/
